<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [0]:
import pandas as pd
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

In [21]:
#Downloading data from the website
col_names = ['party','handicapped_infants','water_project_cost_sharing','adoption_of_the_budget_resolution','physician_fee_freeze',
             'el_salvador_aid','religious_groups_in_schools','anti_satellite_test_ban','aid_to_nicaraguan_contras',
             'mx_missile','immigration','synfuels_corporation_cutback','education_spending',
             'superfund_right_to_sue','crime','duty_free_exports','export_administration_act_south_africa']
voting_data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data",names=col_names)
print(voting_data.shape)
voting_data.head()

(435, 17)


,party,handicapped_infants,water_project_cost_sharing,adoption_of_the_budget_resolution,physician_fee_freeze,el_salvador_aid,religious_groups_in_schools,anti_satellite_test_ban,aid_to_nicaraguan_contras,mx_missile,immigration,synfuels_corporation_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_administration_act_south_africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [24]:
voting_data.dtypes

party                                      object
handicapped_infants                       float64
water_project_cost_sharing                float64
adoption_of_the_budget_resolution         float64
physician_fee_freeze                      float64
el_salvador_aid                           float64
religious_groups_in_schools               float64
anti_satellite_test_ban                   float64
aid_to_nicaraguan_contras                 float64
mx_missile                                float64
immigration                               float64
synfuels_corporation_cutback              float64
education_spending                        float64
superfund_right_to_sue                    float64
crime                                     float64
duty_free_exports                         float64
export_administration_act_south_africa    float64
dtype: object

In [0]:
#Replacing Categoricals values with numbers
voting_data=voting_data.replace({'?':np.nan,'y':1,'n':0})

In [25]:
#Checking for Null values
#Observation : the nulls don't match with the one given on UCI website, I have rechecked the data on UCI seems to be incorrect.
voting_data.isnull().sum()

party                                       0
handicapped_infants                        12
water_project_cost_sharing                 48
adoption_of_the_budget_resolution          11
physician_fee_freeze                       11
el_salvador_aid                            15
religious_groups_in_schools                11
anti_satellite_test_ban                    14
aid_to_nicaraguan_contras                  15
mx_missile                                 22
immigration                                 7
synfuels_corporation_cutback               21
education_spending                         31
superfund_right_to_sue                     25
crime                                      17
duty_free_exports                          28
export_administration_act_south_africa    104
dtype: int64

In [14]:
voting_data.party.value_counts()

democrat      267
republican    168
Name: party, dtype: int64

In [26]:
# Creating Samples based on Parties
republics = voting_data[voting_data.party=='republican']
democrats = voting_data[voting_data.party=='democrat']

print("Republics :",republics.shape)
print("Democrat :",democrats.shape)

Republics : (168, 17)
Democrat : (267, 17)


In [0]:
tstat,pvalue=ttest_ind(republics['handicapped_infants'],democrats['handicapped_infants'],nan_policy='omit')

In [35]:
pvalue

1.613440327937243e-18

In [37]:
#For testing we will omit the null values
#For each of the votes , our null hypothesis is that both the parties vote the same

#Defining function
def stat_significant(x1,x2):
  return ttest_ind(x1,x2,nan_policy='omit')

for col in col_names:
  if(col!='party'):
    tstat,pvalue =stat_significant(republics[col],democrats[col])
    if(pvalue>0.01):
      print("For the issue: ",col,"Demoracts and Republicans does not differ much")
    else:
      print("For the issue: ",col,"Demoracts differ from Repulicans")
  #ttest_ind(republics[col].values,democrats[col].values)

For the issue:  handicapped_infants Demoracts differ from Repulicans
For the issue:  water_project_cost_sharing Demoracts and Republicans does not differ much
For the issue:  adoption_of_the_budget_resolution Demoracts differ from Repulicans
For the issue:  physician_fee_freeze Demoracts differ from Repulicans
For the issue:  el_salvador_aid Demoracts differ from Repulicans
For the issue:  religious_groups_in_schools Demoracts differ from Repulicans
For the issue:  anti_satellite_test_ban Demoracts differ from Repulicans
For the issue:  aid_to_nicaraguan_contras Demoracts differ from Repulicans
For the issue:  mx_missile Demoracts differ from Repulicans
For the issue:  immigration Demoracts and Republicans does not differ much
For the issue:  synfuels_corporation_cutback Demoracts differ from Repulicans
For the issue:  education_spending Demoracts differ from Repulicans
For the issue:  superfund_right_to_sue Demoracts differ from Repulicans
For the issue:  crime Demoracts differ from R